In [6]:
library(DESeq2)
library(EnhancedVolcano)

In [7]:
df <- read.table("dea/sum.tsv", header=TRUE, sep="\t")
# Drop duplicates in id column
df <- df[!duplicated(df$id),]
rownames(df) <- df$id
df$id <- NULL

annotation <- read.table("dea/annotation.bed", header=FALSE, sep="\t", col.names = c("chr", "start", "end", "name", "score", "strand", "type", "gene.id", "gene", "transcripts", "databases"))
# Remove chr, start, end, score, strand, gene.id, transcripts
annotation$chr <- NULL
annotation$start <- NULL
annotation$end <- NULL
annotation$score <- NULL
annotation$strand <- NULL
annotation$gene.id <- NULL
annotation$transcripts <- NULL

rownames(annotation) <- annotation$name
annotation$name <- NULL

phenotype <- read.csv("dea/phenotype.csv", header=TRUE, row.names = 1)
phenotype$transgene <- as.factor(phenotype$transgene)
phenotype$drug <- as.factor(phenotype$drug)

# Center and scale age and induction
phenotype$age <- scale(phenotype$age)
phenotype$induction <- scale(phenotype$induction)

In [8]:
run_analysis <- function(directory, phenotype = NULL, design = NULL, dds = NULL, contrast = NULL, name = NULL) {
    # Fail if both contrast and name are NULL
    if (is.null(contrast) && is.null(name)) {
        stop("Both contrast and name cannot be NULL")
    }
    # Fail if none of contrast and name are NULL
    if (!is.null(contrast) && !is.null(name)) {
        stop("Both contrast and name cannot be provided")
    }

    if (is.null(phenotype) && is.null(design) && is.null(dds)) {
        stop("Either phenotype and design or dds must be provided")
    }

    if (is.null(phenotype) != is.null(design)) {
        stop("Both phenotype and design must be provided")
    }

    if (!is.null(dds) && !is.null(phenotype)) {
        stop("Either dds or phenotype and design must be provided")
    }

    alpha <- 0.05

    if (is.null(dds)) {
        dds <- DESeqDataSetFromMatrix(countData = df[, rownames(phenotype)],
                                    colData = phenotype,
                                    design = design)
        dds <- DESeq(dds)
    }

    if (!is.null(contrast)) {
        res <- results(dds, contrast = contrast, alpha = alpha)
    } else {
        res <- results(dds, name = name, alpha = alpha)
    }
    res <- res[order(res$padj),]
    res <- cbind(res, annotation[rownames(res),])

    dir.create(directory, showWarnings = FALSE)

    EnhancedVolcano(res,
        x='log2FoldChange',
        y='padj',
        lab=res$gene,
        pCutoff = alpha)
    ggsave(paste0(directory, "/volcano.png"))

    # Keep only rows without any NA
    res <- res[complete.cases(res),]

    dir.create("dea/esr1_no", showWarnings = FALSE)

    sign_up <- res[res$log2FoldChange > 0 & res$padj < alpha,]
    upregulated_genes <- unique(unlist(strsplit(sign_up$gene, ",")))
    write.table(upregulated_genes, file=paste0(directory, "/upregulated_genes.txt"), row.names=FALSE, col.names=FALSE, quote=FALSE)
    upregulated_db <- unique(unlist(strsplit(sign_up$databases, ",")))
    write.table(upregulated_db, file=paste0(directory, "/upregulated_db.txt"), row.names=FALSE, col.names=FALSE, quote=FALSE)

    sign_down <- res[res$log2FoldChange < 0 & res$padj < alpha,]
    downregulated_genes <- unique(unlist(strsplit(sign_down$gene, ",")))
    write.table(downregulated_genes, file=paste0(directory, "/downregulated_genes.txt"), row.names=FALSE, col.names=FALSE, quote=FALSE)
    downregulated_db <- unique(unlist(strsplit(sign_down$databases, ",")))
    write.table(downregulated_db, file=paste0(directory, "/downregulated_db.txt"), row.names=FALSE, col.names=FALSE, quote=FALSE)

    return(dds)
}

# Correlation with ESR1 induction

In [9]:
phenotype_esr <- phenotype[phenotype$transgene == "ESR1",]
nrow(phenotype_esr)

[1] 36

## Without drugs

In [10]:
run_analysis(
    "dea/esr1_no",
    phenotype = phenotype_esr[phenotype_esr$drug == "no",],
    design = ~ age + induction,
    name = "induction"
)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Saving 6.67 x 6.67 in image


class: DESeqDataSet 
dim: 29818 30 
metadata(1): version
assays(4): counts mu H cooks
rownames(29818): chr1:3729266-3729444 chr1:4212834-4337843 ...
  chrX_GL456233v2_random:409325-409604
  chrX_GL456233v2_random:409325-422124
rowData names(26): baseMean baseVar ... deviance maxCooks
colnames(30): aging_12m_ESR1_no_1 aging_12m_ESR1_no_2 ...
  antiHormonal_18m_ESR1_no_2 antiHormonal_18m_ESR1_no_3
colData names(7): condition age ... replicate sizeFactor

## Tamoxifen

In [11]:
run_analysis(
    "dea/esr1_tamoxifen",
    phenotype = phenotype_esr[phenotype_esr$drug != "letrozole",],
    design = ~ age + drug + induction,
    contrast = c("drug", "tamoxifen", "no")
)

factor levels were dropped which had no samples

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Saving 6.67 x 6.67 in image


class: DESeqDataSet 
dim: 29818 33 
metadata(1): version
assays(4): counts mu H cooks
rownames(29818): chr1:3729266-3729444 chr1:4212834-4337843 ...
  chrX_GL456233v2_random:409325-409604
  chrX_GL456233v2_random:409325-422124
rowData names(30): baseMean baseVar ... deviance maxCooks
colnames(33): aging_12m_ESR1_no_1 aging_12m_ESR1_no_2 ...
  antiHormonal_18m_ESR1_no_2 antiHormonal_18m_ESR1_no_3
colData names(7): condition age ... replicate sizeFactor

## Letrozole

In [12]:
run_analysis(
    "dea/esr1_letrozole",
    phenotype = phenotype_esr[phenotype_esr$drug != "tamoxifen",],
    design = ~ age + drug + induction,
    contrast = c("drug", "letrozole", "no")
)

factor levels were dropped which had no samples

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Saving 6.67 x 6.67 in image


class: DESeqDataSet 
dim: 29818 33 
metadata(1): version
assays(4): counts mu H cooks
rownames(29818): chr1:3729266-3729444 chr1:4212834-4337843 ...
  chrX_GL456233v2_random:409325-409604
  chrX_GL456233v2_random:409325-422124
rowData names(30): baseMean baseVar ... deviance maxCooks
colnames(33): aging_12m_ESR1_no_1 aging_12m_ESR1_no_2 ...
  antiHormonal_18m_ESR1_no_2 antiHormonal_18m_ESR1_no_3
colData names(7): condition age ... replicate sizeFactor

# Aging

In [13]:
dds_global <- run_analysis(
    "dea/age",
    phenotype = phenotype,
    design = ~ age + transgene + induction + drug,
    name = "age"
)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Saving 6.67 x 6.67 in image


# Drug effects

## Tamoxifen

In [14]:
run_analysis(
    "dea/tamoxifen",
    dds = dds_global,
    contrast=c("drug", "tamoxifen", "no")
)

Saving 6.67 x 6.67 in image


class: DESeqDataSet 
dim: 29818 72 
metadata(1): version
assays(4): counts mu H cooks
rownames(29818): chr1:3729266-3729444 chr1:4212834-4337843 ...
  chrX_GL456233v2_random:409325-409604
  chrX_GL456233v2_random:409325-422124
rowData names(38): baseMean baseVar ... deviance maxCooks
colnames(72): aging_12m_ESR1_no_1 aging_12m_ESR1_no_2 ...
  antiHormonal_18m_ESR1_no_2 antiHormonal_18m_ESR1_no_3
colData names(7): condition age ... replicate sizeFactor

## Letrozole

In [15]:
run_analysis(
    "dea/letrozole",
    dds = dds_global,
    contrast=c("drug", "letrozole", "no")
)

Saving 6.67 x 6.67 in image


class: DESeqDataSet 
dim: 29818 72 
metadata(1): version
assays(4): counts mu H cooks
rownames(29818): chr1:3729266-3729444 chr1:4212834-4337843 ...
  chrX_GL456233v2_random:409325-409604
  chrX_GL456233v2_random:409325-422124
rowData names(38): baseMean baseVar ... deviance maxCooks
colnames(72): aging_12m_ESR1_no_1 aging_12m_ESR1_no_2 ...
  antiHormonal_18m_ESR1_no_2 antiHormonal_18m_ESR1_no_3
colData names(7): condition age ... replicate sizeFactor